In [9]:
# For data processing
import numpy as np
import pandas as pd

# For API usage
import requests as r

# For progress bar
from tqdm import tqdm

In [10]:
API_KEY = '96c7131158ec2d199c338e83be8f0fd5'

# Create empty DF to store the movie data
movie = pd.DataFrame()

resp = r.get("https://api.themoviedb.org/3/discover/movie?" \
                 f"api_key={API_KEY}&language=en-US&sort_by=revenue.desc&" \
                 "include_video=false&page=1")
# Find number of pages in the API call
total_pages = resp.json()['total_pages']

# Relevant movie columns
cols = ['title', 'id', 'vote_average','release_date', 'original_language', 'adult']
# No. of pages to collect responses from
for i in tqdm(range(1, 501)):
        resp = r.get(f"https://api.themoviedb.org/3/discover/movie?api_key={API_KEY}&language=en-US&sort_by=revenue.desc&include_video=false&page={i}")
        
        # Select the relevant columns
        try:
                data = pd.DataFrame(resp.json()['results'],columns=cols)
        except: continue

        movie = movie.append(data)
movie = movie.reset_index(drop=True)

more_cols = ['budget', 'genres', 'revenue', 'decade_released']

rows = []
for i in tqdm(range(len(movie))):
        row = []
        movie_id = movie['id'][i]
        
        
        # Get the movie's other data that is not found in all movies
        resp = r.get(f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={API_KEY}&language=en-US')


        try:
            budget = resp.json()['budget']
            if budget == 0:
                row.append(np.nan)
            else:
                row.append(budget)
        except KeyError:
            row.append(np.nan)
        try:
            genres = []
            for item in resp.json()['genres']:
                genres.append(item['name'])
            row.append(genres)
        except:
            row.append(np.nan)
        
        try:
            revenue = resp.json()['revenue']
            if revenue == 0:
                row.append(np.nan)
            else:
                row.append(revenue)
        except KeyError:
            row.append(np.nan)
        
        try:
            year_movie = int(resp.json()['release_date'][0:4])
            if year_movie >= 2020:
                decade_num = '2020s'
            elif year_movie >= 2010:
                decade_num = '2010s'
            elif year_movie >=2000:
                decade_num = '2000s'
            elif year_movie >=1990:
                decade_num = '1990s'
            elif year_movie >=1980:
                decade_num = '1980s'
            elif year_movie >=1970:
                decade_num = '1970s'
            elif year_movie >=1960:
                decade_num = '1960s'
            else:
                decade_num = 'Before 1960s'
            row.append(decade_num)
        except:
            row.append(np.nan)
        

        rows.append(row)

# Create a DF containing all the data for all movies
stats = pd.DataFrame(rows, columns=more_cols)

df = pd.concat([movie, stats], axis=1)

df


  0%|          | 0/500 [00:00<?, ?it/s]/var/folders/18/95745h4n0hl94h03fr01c1jc0000gn/T/ipykernel_17786/3457398657.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movie = movie.append(data)
  0%|          | 1/500 [00:00<01:20,  6.21it/s]/var/folders/18/95745h4n0hl94h03fr01c1jc0000gn/T/ipykernel_17786/3457398657.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movie = movie.append(data)
  0%|          | 2/500 [00:00<02:45,  3.01it/s]/var/folders/18/95745h4n0hl94h03fr01c1jc0000gn/T/ipykernel_17786/3457398657.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movie = movie.append(data)
  1%|          | 3/500 [00:00<02:51,  2.90it/s]/var/folders/18/95745h4n0hl94h03fr01c1jc0000gn/T/ipykernel_17786/3457398657.py:23: Fu

KeyboardInterrupt: 

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   title              40 non-null     object 
 1   id                 40 non-null     int64  
 2   vote_average       40 non-null     float64
 3   release_date       40 non-null     object 
 4   original_language  40 non-null     object 
 5   adult              40 non-null     bool   
 6   budget             40 non-null     int64  
 7   genres             40 non-null     object 
 8   revenue            40 non-null     int64  
 9   decade_released    38 non-null     object 
 10  keywords           39 non-null     object 
dtypes: bool(1), float64(1), int64(3), object(6)
memory usage: 3.3+ KB


In [ ]:
df.to_csv('uncleaned_1015.csv')